In [30]:
import sys
import time
import csv
import json
import pandas as pd
from pytrends.request import TrendReq

#%%


# 사용자 맞춤 변수 지정
numb = 2 # 상위부터 추출 개수 
lang = 'ko' # pytrend 기준 언어 설정
time = 540 # pytrend 기준 시간대 설정
geo = 'KR' # pytrend 기준 위치 설정
month = 1 # 현재부터 n개월간의 기록 (정수만 입력)
update = 1 # 업데이트할 주기(단위 sec)  # 86400 하루
address = "C:/Users/sbeen/OneDrive/바탕 화면/keyword_data.csv"
dicts = 7 # 웹에 띄워지는 초기값
# self.jsons = 0


lines = [] # 유사키워드별 리스트 분류
keys = [] # 유사키워드 중 대표 키워드만 모음
sums = [] # 키워드에 대한 총 합산 값


def read_csv():

    with open(address, "r") as file:
        reader = csv.reader(file)
        print(reader,'\n')
        for line in reader:
            line = [field for field in line if field != ''] # 최대 5개 입력인데, 미만일 경우 발생하는 공백제거
            lines.append(line) # 한 줄에 해당하는 유사키워드별로 묶기
            if line:
                keys.append(line[0]) # 맨 첫 번째를 대표키워드로 지정
        print(lines,'\n')
        print(keys,'\n')

def py_trends():

    pytrends = TrendReq(hl=lang, tz=time) # pytrend 언어, 시간대 설정
    for i in range(len(lines)): # 키워드별 pytrend 데이터 수집 반복
        pytrends.build_payload(lines[i], cat=0, timeframe='today %d-m'%month, geo=geo) # 최신 3달치
        data = pytrends.interest_over_time() 
        print(data)
        data = data.reset_index() 
        data = data.iloc[:, 1:-1] # 필요없는 부분 date_time 열 제거
        print(data)
        sums.append(int(data.sum().sum())) # 유사키워드 모두 합한 값
        print(sums)


def to_json():
    # dicts = dict(zip(self.keys, self.sums)) # list 두 개를 각각 key, value로 

    df = pd.DataFrame({'keys': keys, 'sums': sums})
    print('\n유사키워드에 대한 검색량 총합 :\n', df)
    df = df.sort_values(by='sums', ascending=False) # 내림차순
    print('\n키워드별 정렬:\n', df)
    df = df[:numb]
    print('\n상위 n개 추출:\n', df)
    df = df.set_index('keys').T 
    dicts = df.to_dict('records')[0] #orient = 'records'임
    print('\nGET에 사용될 정보 :', dicts)
    jsons = json.dumps(dicts) # dict -> json
    return jsons # get_data에서 참조하는 값 즉 data.jsons'


def main():
    try:
        while True:
            print("2. reading csv file\n")
            read_csv()
            print("3. loading a google trend data\n")
            py_trends()
            print("4. making a json file")
            print('\nJson으로 변경된 정보: ', to_json(),'\n')
            print("5. waiting.. It takes a day to update the pytrend data\n\n\n")
            time.sleep(update)
            print("1. program restart!\n")
    except KeyboardInterrupt:
            print("Program interrupted by user.")
            sys.exit(0)

# 1. read_csv()

In [31]:
read_csv()


[['파이썬', 'python', 'py', 'PY', 'PYTHON'], ['자바', 'JAVA', 'java'], ['c++', '씨쁠쁠'], ['C', 'C언어']] 

['파이썬', '자바', 'c++', 'C'] 



# 2. py_trends()

In [32]:
py_trends()

            파이썬  python  py  PY  PYTHON  isPartial
date                                              
2023-01-10   57      87   3   3      87      False
2023-01-11   53      80   0   0      80      False
2023-01-12   53      74   0   0      74      False
2023-01-13   49      73   2   2      73      False
2023-01-14   33      34   0   0      34      False
2023-01-15   37      46   2   2      46      False
2023-01-16   58      95   1   1      95      False
2023-01-17   72     100   1   1     100      False
2023-01-18   53      76   1   1      76      False
2023-01-19   53      80   2   2      80      False
2023-01-20   54      59   0   0      59      False
2023-01-21   31      31   0   0      31      False
2023-01-22   23      25   0   0      25      False
2023-01-23   33      31   2   2      31      False
2023-01-24   34      34   0   0      34      False
2023-01-25   53      77   2   2      77      False
2023-01-26   55      78   0   0      78      False
2023-01-27   54      81   3   3

# 3. to_json()

In [34]:
to_json()


유사키워드에 대한 검색량 총합 :
   keys  sums
0  파이썬  4996
1   자바  5858
2  c++  2307
3    C  2656

키워드별 정렬:
   keys  sums
1   자바  5858
0  파이썬  4996
3    C  2656
2  c++  2307

상위 n개 추출:
   keys  sums
1   자바  5858
0  파이썬  4996

GET에 사용될 정보 : {'자바': 5858, '파이썬': 4996}


'{"\\uc790\\ubc14": 5858, "\\ud30c\\uc774\\uc36c": 4996}'